In [1]:
%matplotlib inline

In [5]:
from kaggle_quora_question_pairs_common import *

dataset.hdf
sample_submission.csv
sample_submission.csv.zip
test.csv
test.csv.zip
train.csv
train.csv.zip



/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
train_df, test_df = load_train_test()
train_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
log_max_mem_usage()
train_df.shape

Current all-time max memory: 859 MB


(404290, 6)

In [27]:
%%time

include_test = True
unique_questions = get_unique_questions(train_df, test_df, include_test=include_test)

log_max_mem_usage()

Current all-time max memory: 13089 MB
CPU times: user 976 ms, sys: 8 ms, total: 984 ms
Wall time: 980 ms


In [38]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


stops = stopwords.words('english')
morphy_tag = {
    'NN': wordnet.NOUN,
    'JJ': wordnet.ADJ,
    'VB': wordnet.VERB,
    'VBN': wordnet.VERB,
    'RB': wordnet.ADV,
    'RBR': wordnet.ADV
}


def get_wordnet_pos(treebank_tag, default=wordnet.NOUN):
#     http://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
#     http://stackoverflow.com/questions/7706696/how-can-i-best-determine-the-correct-capitalization-for-a-word
    tag = default
    
    if treebank_tag.startswith('J'):
        tag = wordnet.ADJ
    elif treebank_tag.startswith('V'):
        tag = wordnet.VERB
    elif treebank_tag.startswith('N'):
        tag = wordnet.NOUN
    elif treebank_tag.startswith('R'):
        tag = wordnet.ADV
    
    return tag


def get_lem_tag(tag):
    return get_wordnet_pos(tag, default=wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

def lemmatize_word(word_tag):
    word, tag = word_tag
    tag = get_lem_tag(tag)
    return lemmatizer.lemmatize(word, pos=tag)


def lemmatize_words(words_tags):
    lemmas = []
    for word_tag in words_tags:
        lemmas.append(lemmatize_word(word_tag))
        
    return lemmas

In [ ]:
corpus = .str.replace("'", '').str.replace('\W', ' ').str.lower().dropna()
lemmatized_corpus = corpus.str.split().map(lambda x: lemmatize_words(pos_tag(x)))

words = [
    word for response in lemmatized_corpus for word in response if word not in stops
]


In [251]:
import re


SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}

def clean_string(text):
    
    def pad_str(s):
        return ' ' + s + ' '
    
    # Empty question
    
    if type(text) != str or text=='':
        return ''

    # preventing first and last word being ignored by regex    
    text = ' ' + text + ' '

    # Replace weird chars in text
    
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("？", "?", text) 
    text = re.sub("…", " ", text) 
    text = re.sub("é", "e", text) 
    
    # Clean shorthands
    
    text = re.sub("\'s ", " is ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub(" i'm ", " I am ", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub(r"(\W|^)([0-9]+)[kK](\W|$)", r"\1\g<2>000\3", text) # better regex provided by @armamut
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
#     # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
#     text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
#     # all numbers should separate from words, this is too aggressive
    
#     def pad_number(pattern):
#         matched_string = pattern.group(0)
#         return pad_str(matched_string)
#     text = re.sub('[0-9]+', pad_number, text)
    
    # add padding to punctuations and special chars, we still need them later
    
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    def pad_pattern(pattern):
        matched_string = pattern.group(0)
        return pad_str(matched_string)
    text = re.sub('''[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\\'\"]''', pad_pattern, text) 
        
#     text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii word with special word
    text = text.decode('utf-8')
    
    # indian dollar
    
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)
    
    # clean text rules get from : https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" banglore ", " Banglore ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    
    # typos identified with my eyes
    
    text = re.sub(r" quikly ", " quickly ", text)
    text = re.sub(r" unseccessful ", " unsuccessful ", text)
    text = re.sub(r" demoniti[\S]+ ", " demonetization ", text, flags=re.IGNORECASE)
    text = re.sub(r" demoneti[\S]+ ", " demonetization ", text, flags=re.IGNORECASE)  
    text = re.sub(r" addmision ", " admission ", text)
    text = re.sub(r" insititute ", " institute ", text)
    text = re.sub(r" connectionn ", " connection ", text)
    text = re.sub(r" permantley ", " permanently ", text)
    text = re.sub(r" sylabus ", " syllabus ", text)
    text = re.sub(r" sequrity ", " security ", text)
    text = re.sub(r" undergraduation ", " undergraduate ", text) # not typo, but GloVe can't find it
    text = re.sub(r"(?=[a-zA-Z])ig ", "ing ", text)
    text = re.sub(r" latop", " laptop", text)
    text = re.sub(r" programmning ", " programming ", text)  
    text = re.sub(r" begineer ", " beginner ", text)  
    text = re.sub(r" qoura ", " Quora ", text)
    text = re.sub(r" wtiter ", " writer ", text)  
    text = re.sub(r" litrate ", " literate ", text)  

    # the single 's' in this stage is 99% of not clean text, just kill it
    text = re.sub(' s ', " ", text)
    
    # reduce extra spaces into single spaces
    text = re.sub('[\s]+', " ", text)
    text = text.strip()
    
    return text


def clean_tokenize_lemmatize(text, return_tokens=True):
    tokens = lemmatize_words(pos_tag(word_tokenize(clean_string(text))))
    
    if not return_tokens:
        tokens = ' '.join(tokens)
    
    return tokens

In [275]:
batch_size = 5000
partitions = (unique_questions.shape[0] + batch_size) / batch_size
partitions

958

In [ ]:
%%time
print 'Total expected partitions to process: {}'.format(partitions)

for ix, batch in enumerate(np.array_split(unique_questions, partitions)):
    batch.map(lambda x: clean_tokenize_lemmatize(x, False)).to_csv(
        'corpus/lemmatized-unnormalized-cleaned-data.with_test.csv', index=False, mode='a', sep='\n',
        encoding='utf-8'
    )
    print 'Processed partition: {}'.format(ix + 1)

Total expected partitions to process: 958
Processed partition: 1
Processed partition: 2
Processed partition: 3
Processed partition: 4
Processed partition: 5
Processed partition: 6
Processed partition: 7
Processed partition: 8
Processed partition: 9
Processed partition: 10
Processed partition: 11
Processed partition: 12
Processed partition: 13
Processed partition: 14
Processed partition: 15
Processed partition: 16
Processed partition: 17
Processed partition: 18
Processed partition: 19
Processed partition: 20
Processed partition: 21
Processed partition: 22
Processed partition: 23
Processed partition: 24
Processed partition: 25
Processed partition: 26
Processed partition: 27
Processed partition: 28
Processed partition: 29
Processed partition: 30
Processed partition: 31
Processed partition: 32
Processed partition: 33
Processed partition: 34
Processed partition: 35
Processed partition: 36
Processed partition: 37
Processed partition: 38
Processed partition: 39
Processed partition: 40
Process

In [13]:
%%time
from gensim.models import KeyedVectors

wvmodel = KeyedVectors.load_word2vec_format(
    '/home/avsolatorio/WORK/kaggle/pre-trained-models/GoogleNews-vectors-negative300.bin.gz', binary=True
)

CPU times: user 1min 7s, sys: 972 ms, total: 1min 8s
Wall time: 1min 8s


In [101]:
import enchant

In [102]:
d = enchant.Dict('en-us')

In [145]:
d.check('bedraggled')

True